# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project. 

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [1]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /private/var/folders/7n/47sl211117v2958gd5nysr5r0000gn/T/pip-req-build-bd7s3r71
  Running command git clone -q git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /private/var/folders/7n/47sl211117v2958gd5nysr5r0000gn/T/pip-req-build-bd7s3r71
  Resolved git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to commit bcb2d9c1b1ef39816b750912647fdf91f3d23fbc
     |████████████████████████████████| 5.6 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 117 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 180 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 46 kB 7.2 MB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████

     |████████████████████████████████| 5.0 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 609 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 450 kB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 333 kB/s eta 0:00:01
     |████████████████████████████████| 106 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 21.5 MB/s eta 0:00:01
  Created wheel for lucem-illud: filename=lucem_illud-8.0.1-py3-none-any.whl size=34963 sha256=bf0906357fd6fdbcc1d4b2d4e7d0fb3848d0ce44371437d25a4e61f2e70250b4
  Stored in directory: /private/var/folders/7n/47sl211117v2958gd5nysr5r0000gn/T/pip-ephem-wheel-cache-71k52wmc/wheels/26/66/59/2c69dc2c9856ccf2d3eb274e5754cbb361d91e0a577f63c61d
  Created wheel for pyanno3: filename=pyanno3-2.0.2-py3-none-any.whl size=116978 sha256=8ce66aee042526bebc37eb877fee

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

For this notebook we will be using the following packages:

In [3]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [3]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [6]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [7]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"296cc375-e49f-4836-b061-a9b1aa860fa5","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":1065940520,"wgRevisionId":1065940520,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description matches Wikidata","Articles prone to spam from October 2017","Articles with BNE identifiers",

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [12]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [13]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia









































Content analysis

From Wikipedia, the free encyclopedia



Jump to navigation
Jump to search
Research method for studying docu


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [11]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [14]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

South Asia 

Middle East

Europe



Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [15]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0                                       South Asia \n
1                                       Middle East\n
2                                            Europe\n
3                                     North America\n
4   Content analysis is the study of documents and...
5   Practices and philosophies of content analysis...
6   Computers are increasingly used in content ana...
7   Content analysis is best understood as a broad...
8   The simplest and most objective form of conten...
9   A further step in analysis is the distinction ...
10  Quantitative content analysis highlights frequ...
11  Siegfried Kracauer provides a critique of quan...
12  With the rise of common computing facilities l...
13  By having contents of communication available ...
14  Computer-assisted analysis can help with large...
15  Robert Weber notes: "To make valid inferences ...
16  There are five types of texts in content analy...
17  Content analysis is rese

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [16]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [17]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [18]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color="red">Exercise 1</font>
<font color="red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

In [55]:
exercise_1_url = 'https://www.vox.com/recode/22836368/amazon-antitrust-ftc-marketplace'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62'}

In [6]:
exercise_1_response = requests.get(exercise_1_url, headers = headers)

In [9]:
e1soup = bs4.BeautifulSoup(exercise_1_response.text)

In [14]:
e1title = e1soup.find('h1', class_ = 'c-page-title').get_text()
e1title.strip()

'The true cost of Amazon’s low prices'

In [16]:
e1author = e1soup.find('span', class_ = 'c-byline__author-name').get_text()
e1author

'Sara Morrison'

In [19]:
e1time = e1soup.find('time', {'data-ui': 'timestamp'}).get_text()
e1time.strip()

'Jan 13, 2022,  9:00am EST'

In [21]:
e1sub = e1soup.find('p', class_ = 'c-entry-summary p-dek').get_text()
e1sub

'Critics say the “everything store” does too much. Is 2022 the year antitrust hawks come for Amazon?'

In [24]:
e1par = e1soup.find('div', class_ = 'c-entry-content').get_text()
print(e1par)



This story is part of a Recode series about Big Tech and antitrust. Over the next few weeks, we’ll cover what’s happening with Apple, Amazon, Facebook, Google, and Microsoft.
On the heels of yet another year of record sales, Amazon is dealing with a couple of unwelcome updates in the new year. The Senate Judiciary Committee has announced it will soon be marking up the American Innovation and Choice Online Act, an antitrust bill targeting Amazon and other Big Tech companies. This follows reports that the Federal Trade Commission is ramping up its years-long antitrust investigation into Amazon’s cloud computing arm, Amazon Web Services, or AWS. 
It’s clearer now than ever that Amazon, which was allowed to grow mostly unhindered for more than two decades, is caught in the middle of an international effort to check Big Tech’s power.
The Senate bill, one of several bipartisan antitrust bills in Congress, would prohibit Amazon from giving its products preferential treatment, among other th

In [109]:
# regex 1: quotes
quotes = re.findall(r'\“.+\”', e1par)
quotes


['“Amazon’s Antitrust Paradox.”',
 '“is best suited to understand the various issues and problems with Amazon,” said Alex Harman, a competition policy advocate at Public Citizen, a consumer advocacy group. “And we are very excited that she will be able to bring a significant action against them.”',
 '“intense competition” in all of its lines of business. It says its expansion is part of a long-running strategy to make “big bets over the long term to reinvent the customer experience.”',
 '“Amazon leverages its power in one space to take over a new space, which is core to their business practice. They have the ability to combine the competitive advantages of different aspects of their business to take over new sectors of the economy.”',
 '“That’s where there’s a lot of obvious harms, and where you have businesses who are unhappy with how they’re being treated,”',
 '“everything store”',
 '“There are dynamics in digital that are fundamentally different,” Andrew Lipsman, principal analyst a

In [104]:
# regex 2: acronyms
acronyms = re.findall(r'([A-Z]{2,})', e1par)
acronyms

['AWS',
 'FTC',
 'AWS',
 'FTC',
 'FTC',
 'FTC',
 'FTC',
 'FTC',
 'AWS',
 'MGM',
 'US',
 'DC',
 'DC',
 'DC',
 'DC',
 'FBA',
 'FBA',
 'AFP',
 'FBA',
 'FBA',
 'DC',
 'FTC',
 'FTC',
 'DC']

In [101]:
# regex 3: names
names = re.findall(r'([A-Z][\w-]*(\s+[A-Z][\w-]*)+)', e1par)
names


[('Big Tech', ' Tech'),
 ('The Senate Judiciary Committee', ' Committee'),
 ('American Innovation', ' Innovation'),
 ('Choice Online Act', ' Act'),
 ('Big Tech', ' Tech'),
 ('Federal Trade Commission', ' Commission'),
 ('Amazon Web Services', ' Services'),
 ('Big Tech', ' Tech'),
 ('The Senate', ' Senate'),
 ('But Amazon', ' Amazon'),
 ('The FTC', ' FTC'),
 ('Big Tech', ' Tech'),
 ('Lina Khan', ' Khan'),
 ('Yale Law Journal', ' Journal'),
 ('Antitrust Paradox', ' Paradox'),
 ('Alex Harman', ' Harman'),
 ('Public Citizen', ' Citizen'),
 ('Lina Khan', ' Khan'),
 ('Federal Trade Commission', ' Commission'),
 ('An Rong Xu', ' Xu'),
 ('Washington Post', ' Post'),
 ('Getty Images\n\n\nKhan', '\n\n\nKhan'),
 ('Sarah Miller', ' Miller'),
 ('American Economic Liberties Project', ' Project'),
 ('How Amazon', ' Amazon'),
 ('United States', ' States'),
 ('United States', ' States'),
 ('Amazon Prime', ' Prime'),
 ('Justin Sullivan', ' Sullivan'),
 ('Getty Images\n\n\nSmaller', '\n\n\nSmaller'),
 ('

In [115]:
# regex 4: parentheses
parentheses = re.findall(r'\(.+\)', e1par)
parentheses


['(or even several markets)',
 '(and Amazon’s attempt to buy MGM)',
 '(Amazon says it’s standard practice for retailers to use data about customers’ interests to help determine what to make for their own private labels.)',
 '(Amazon wouldn’t comment)']

In [131]:
# regex 5: hyphenated words
hyphen = re.findall(r'(\w+(\-\w+){1,})', e1par)
hyphen

[('years-long', '-long'),
 ('anti-competitive', '-competitive'),
 ('long-term', '-term'),
 ('third-party', '-party'),
 ('non-public', '-public'),
 ('long-running', '-running'),
 ('anti-monopoly', '-monopoly'),
 ('third-party', '-party'),
 ('e-commerce', '-commerce'),
 ('second-largest', '-largest'),
 ('e-commerce', '-commerce'),
 ('catch-up', '-up'),
 ('medium-sized', '-sized'),
 ('third-party', '-party'),
 ('e-commerce', '-commerce'),
 ('first-party', '-party'),
 ('much-touted', '-touted'),
 ('third-party', '-party'),
 ('Self-Reliance', '-Reliance'),
 ('third-party', '-party'),
 ('anti-competitive', '-competitive'),
 ('third-party', '-party'),
 ('non-public', '-public'),
 ('self-preferencing', '-preferencing'),
 ('Self-preferencing', '-preferencing'),
 ('catch-all', '-all'),
 ('anti-competitive', '-competitive'),
 ('self-preferencing', '-preferencing'),
 ('Tech-focused', '-focused'),
 ('first-party', '-party'),
 ('resource-intensive', '-intensive')]

In [31]:
e1df = pandas.DataFrame(data = {'time': e1time.strip(),
                         'author': e1author,
                         'title': e1title.strip(),
                         'subtitle': e1sub,
                         'content': e1par},
                index = [1])
e1df

,time,author,title,subtitle,content
1,"Jan 13, 2022, 9:00am EST",Sara Morrison,The true cost of Amazon’s low prices,Critics say the “everything store” does too mu...,\n\nThis story is part of a Recode series abou...


In [44]:
def add_data(url):
    '''
    Add a row of data to current dataframe
    '''
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text)
    title = soup.find('h1', class_ = 'c-page-title').get_text().strip()
    author = soup.find('span', class_ = 'c-byline__author-name').get_text()
    time = soup.find('time', {'data-ui': 'timestamp'}).get_text().strip()
    sub = soup.find('p', class_ = 'c-entry-summary p-dek').get_text()
    par = soup.find('div', class_ = 'c-entry-content').get_text()
    new_df = pandas.DataFrame(data = {'time': time,
                            'author': author,
                            'title': title,
                            'subtitle': sub,
                            'content': par},
                             index = [1])
    return new_df

In [45]:
new_df = add_data('https://www.vox.com/the-goods/22880345/pandemic-shopper-convenience-instant-delivery')

In [46]:
new_df

,time,author,title,subtitle,content
1,"Jan 13, 2022, 7:00am EST",Terry Nguyen,The empty promise of instant delivery,The pandemic changed how shoppers think about ...,"\nAs a resident of New York City, I face long ..."


In [47]:
e1df.append(new_df)

,time,author,title,subtitle,content
1,"Jan 13, 2022, 9:00am EST",Sara Morrison,The true cost of Amazon’s low prices,Critics say the “everything store” does too mu...,\n\nThis story is part of a Recode series abou...
1,"Jan 13, 2022, 7:00am EST",Terry Nguyen,The empty promise of instant delivery,The pandemic changed how shoppers think about ...,"\nAs a resident of New York City, I face long ..."



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [ ]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

In [ ]:
print(contentPTags)

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [ ]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [ ]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [ ]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF


# <font color="red">Exercise 2</font>
<font color="red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



In [171]:
def recursive_keyword(max_loops, starting_url):
    for loop_num in max_loops:
        response = requests.get(starting_url)
            links = re.findall(r'<a\s+href=(?:"([^"]+)"|'([^']+)').*?>(.*?)</a>', response.text)
        recursive_keyword()
    return

IndentationError: unexpected indent (4035366103.py, line 4)

In [141]:
response = requests.get('https://www.youtube.com/watch?v=tsZCoJMoxv4&ab_channel=%E9%99%B3%E5%AF%97NingSelect')

In [142]:
soup = bs4.BeautifulSoup(response.text, parser = 'html.parser')

In [ ]:
url_lst = recursive_keyword(10, 'https://www.youtube.com/results?search_query=%E5%9C%8B%E6%96%87%E4%BD%9C%E6%96%87')
url_lst

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [29]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3925,"posts-type":false,"posts":[{"id":"662815854023655425","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/662815854023655425","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/662815854023655425","type":"photo","date-gmt":"2021-09-20 04:00:56 GMT","date":"Mon, 20 Sep 2021 06:00:56","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1632110456,"format":"html","reblog-key":"kjH8cg34","slug":"","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"662815854023655425\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_662815854023655425\"><iframe id=\"like_iframe_662815854023655425\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=lolcats-lol-cat&amp;post_id=66281585402365

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [30]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [31]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [32]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,662815854023655425,https://64.media.tumblr.com/021eac8fbcafbb00a5...,"Mon, 20 Sep 2021 06:00:56","[gif, lolcat, lolcats, cat, funny]",gif
1,662778109891952640,https://64.media.tumblr.com/8c0517adb8c71e4a3d...,"Sun, 19 Sep 2021 20:01:00","[cat, cats, lol, lolcat, lolcats]",png
2,662657302700146688,https://64.media.tumblr.com/061d27cda309d5c809...,"Sat, 18 Sep 2021 12:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
3,662513901538246656,https://64.media.tumblr.com/80584a9d1ff4ddc4fc...,"Thu, 16 Sep 2021 22:01:32","[cat, cats, lol, lolcat, lolcats]",png
4,662257177983090688,https://64.media.tumblr.com/893b320cd2e8970a20...,"Tue, 14 Sep 2021 02:01:01","[cat, cats, lol, lolcat, lolcats]",jpg
5,662166591527698432,https://64.media.tumblr.com/c7f0a0a9184e480e15...,"Mon, 13 Sep 2021 02:01:11","[cat, cats, lol, lolcat, lolcats]",jpg
6,662113740899090432,https://64.media.tumblr.com/07f7be7f71917a6049...,"Sun, 12 Sep 2021 12:01:09","[cat, cats, lol, lolcat, lolcats]",jpg
7,661955166248026112,https://64.media.tumblr.com/205f030c48d31f8960...,"Fri, 10 Sep 2021 18:00:40","[cat, cats, lol, lolcat, lolcats]",jpg
8,661894830378614784,https://64.media.tumblr.com/c463bff883fec2045b...,"Fri, 10 Sep 2021 02:01:39","[cat, cats, lol, lolcat, lolcats]",png
9,661864596682964992,https://64.media.tumblr.com/b840ddb3aa2303206b...,"Thu, 09 Sep 2021 18:01:06","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [33]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [34]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [37]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [39]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [40]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
phoenix_and_the_turtle.txt,THE PHOENIX AND THE TURTLE\n\nby William Shake...
king_lear.txt,The Tragedie of King Lear\n\n\nActus Primus. S...
passionate_pilgrim.txt,THE PASSIONATE PILGRIM\n\nby William Shakespea...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
two_gentlemen_of_verona.txt,THE TWO GENTLEMEN OF VERONA\n\nby William Shak...
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [41]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [151]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [43]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [44]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [45]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [46]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [47]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [48]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="red">Exercise 3</font>
<font color="red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


In [153]:
pdfurl = 'https://dl.acm.org/doi/pdf/10.1145/2187836.2187931?casa_token=38JurJdWVWwAAAAA:u-6cj2VBVHdDef5xPQHHbmhMcZSPNeP2jFeklzOcqgdlj_zfeNebyNDM90JGZr-LtjDKCVqN61iy8A'
pdfrequest = requests.get(pdfurl, stream=True)

In [157]:
pdfbytes = io.BytesIO(pdfrequest.content)
pdfstring = readPDF(pdfbytes)
print(pdfstring)

Echoes of Power: Language Effects and Power Differences

in Social Interaction

Cristian Danescu-Niculescu-Mizil

Cornell University

cristian@cs.cornell.edu,

llee@cs.cornell.edu,

bopang@yahoo-inc.com,

Lillian Lee

Cornell University

Bo Pang
Yahoo!

Jon Kleinberg
Cornell University
kleinber@cs.cornell.edu

ABSTRACT
Understanding social interaction within groups is key to analyz-
ing online communities. Most current work focuses on structural
properties: who talks to whom, and how such interactions form
larger network structures. The interactions themselves, however,
generally take place in the form of natural language — either spo-
ken or written — and one could reasonably suppose that signals
manifested in language might also provide information about roles,
status, and other aspects of the group’s dynamics. To date, how-
ever, ﬁnding domain-independent language-based signals has been
a challenge.

Here, we show that in group discussions, power differentials be-
tween participants

In [169]:
#title, author, url, full text
filename = pdfstring[:10]
pdfdf = pandas.DataFrame({'filename': filename, 'url': pdfurl, 'content': pdfstring}, index = [1])


In [170]:
pdfdf

,filename,url,content
1,Echoes of,https://dl.acm.org/doi/pdf/10.1145/2187836.218...,Echoes of Power: Language Effects and Power Di...


# <font color="red">Exercise 4</font>

<font color="red">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.] 

## ***What?*** 
<100 words

I want to use the pushshift Reddit data to analyze how academic ideas are used in online discourse. I want to understand how and when users (actors) in online environments (world) are inclined to invoke academic ideas, whether it is empirical research, theories, or media appearances, in order to speak with more weight in online discussions (game).

## ***Why?***
<150 words

The advantage in using pushshift Reddit data is manifold. First, compared with using the Reddit API, the data in pushshift is more readily accessible. Second, Reddit, as compared with Facebook, has better data access in general. Third, compared with Twitter, the platform affordances of Reddit allows for more elaborate and wordy political discussions.

The disadvantages in using pushshift Reddit data is it's scale: since pushshift data is mainly accessible through batch downloads, it's harder to access only a subsection of all the content available. Pushshift data is also only a fraction of all the online discussions, and may contain a combination of platform, user, and accessibility bias. For example, according to [this site](https://www.statista.com/statistics/1255182/distribution-of-users-on-reddit-worldwide-gender/), Reddit users are composed of 63% male and 37% female.

Alternative sources for online discussion using academic ideas may come from various question-and-answer forums, such as Quora, StackOverflow, and 知乎.

### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

